---
title: "Zig-Zag 過程によるサンプリング"
subtitle: "ジャンプと確定的な動きによる新たな MCMC 手法"
author: "司馬博文"
date: 7/3/2024
date-modified: 7/18/2024
categories: [Process, Sampling, Julia, MCMC]
bibliography: 
    - ../../../assets/2023.bib
    - ../../../assets/2024.bib
    - ../../../assets/2025.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: Zig-Zag サンプラー定義とエルゴード性を解説する．続いて，Zig-Zag サンプラーは非対称なダイナミクスを持つために，従来の MCMC よりも速い収束が期待されることを，MALA との比較でみる．最後に，Zig-Zag サンプラーの実装に用いたパッケージとその利用方法を示す．
code-fold: false
execute:
    cache: true
listing: 
    -   id: lst-pdmp
        type: grid
        grid-columns: 1
        image-height: 100%
        contents:
            - "../Process/PureJump.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
    -   id: lst-zigzag
        type: grid
        grid-columns: 1
        image-height: 100%
        contents:
            - "../Stat/ZigZagSubsampling.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
    -   id: lst-julia
        type: grid
        grid-columns: 1
        image-height: 100%
        contents:
            - "../Julia/MCMCwithJulia.qmd"
        date-format: iso
        fields: [title,image,date,subtitle]
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\operatorname{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



## Zig-Zag 過程

### はじめに

１次元の Zig-Zag 過程は元々，[Curie-Weiss 模型](../Nature/StatisticalMechanics1.qmd#sec-Curie-Weiss) における Glauber 動力学を lifting により非可逆化して得る Markov 連鎖の，スケーリング極限として特定された Feller-Dynkin 過程である [@Bierkens-Roberts2017]．

区分確定的 Markov 過程（PDMP）といい，ランダムな時刻にランダムな動きをする以外は，決定論的な動きをする過程である．

![$\R^2$ 上の Gauss 分布に収束する Zig-Zag 過程の軌跡](ZigZag2D_Gaussian.svg)

PDMP の一般論については次の記事も参照：

::: {#lst-pdmp}
:::

ただし Zig-Zag 過程は，[@Goldstein1951] で電信方程式と関連して，同様の過程が扱われた歴史もある．

### 設定

Zig-Zag 過程 $Z=(X,\Theta)$ の状態空間は，力学的な立場に立って $E=\R^d\times\{\pm1\}^d$ と見ることが多い．

力学における[相空間](https://ja.wikipedia.org/wiki/相空間)と同様，$X\in\R^d$ が位置を，$\theta\in\{\pm1\}^d$ が速度を表すと解する．すなわち，Zig-Zag 過程は全座標系と $45$ 度をなす方向に，常に一定の単位速さで $\R^d$ 上を運動する粒子とみなせる．

すなわち，$(x,\theta)\in E$ から出発する Zig-Zag 過程は，次の微分方程式系で定まる決定論的なフロー $\phi_{(x,\theta)}:\R\to\R^d$ に従って運動する粒子とみなせる：
$$
\dd{\phi_{(x,\theta)}(t)}{t}=\theta,\qquad \dd{\Theta_t}{t}=0.
$$

### アルゴリズム {#sec-algorithm}

#### 全体像

Zig-Zag 過程 $Z$ は次のようにしてシミュレーションできる：

::: {.callout-tip appearance="simple" icon="false" title="Zig-Zag 過程のシミュレーション"}

1. レート関数 $\lambda_1,\cdots,\lambda_d$ から定まる強度関数
    $$
    m_i(t):=\lambda_i(x+\theta t,\theta),\qquad i\in[d],
    $$
    を持つ，$d$ 個の独立な $\R_+$ 上の非一様 Poisson 点過程の，最初の到着時刻 $T_1,\cdots,T_d$ をシミュレーションする．
2. 最初に到着した座標番号 $j:=\argmin_{i\in[d]}T_i$ について，時刻 $T_j$ に速度成分 $\theta_j$ の符号を反転させる．すなわち，関数
    $$
    F_j(\theta)_i:=\begin{cases}-\theta_i&i=j\\\theta_i&i\ne j\end{cases}
    $$
    に従ってジャンプする．
3. １に $t=T_j$ として戻って，くり返す．

:::
<!-- 多次元の Poisson 過程の各成分の跳躍は独立だから，^[[@Revuz-Yor1999 p.473] 命題XII.1.7．] それぞれの成分ごとに Poisson 到着時刻 $T_i\;(i\in[d])$ をシミュレーションし，最初に到着したものを $T_j$ とすると，関数
$$
F_j(\theta)_i=\begin{cases}-\theta_i&i=j\\\theta_i&i\ne j\end{cases}
$$
に従ってジャンプすると考えて良い． -->

::: {.callout-important title="Zig-Zag 過程の跳躍測度" icon="false" collapse="true"}

もう一つ，MCMC の文脈で自然な見方は，状態空間を
$$
E=\bigcup_{\theta\in\{\pm1\}^d}\R^d\times\{\theta\}
$$
と取る見方である．これは [@Davis1993] による [一般の PDMP の設定](PureJump.qmd#sec-settings) と対応する．

この $E$ 上で，レート関数
$$
\lambda(x,\theta):=\sum_{i=1}^d\lambda_i(x,\theta)
$$
が定める強度
$$
M(t):=\lambda(x+t\theta,\theta)
$$
を持った $\R_+$ 上の非一様 Poisson 点過程に従ってジャンプが訪れる．

この点過程に対して，確率核
$$
Q((x,\theta),-):=\sum_{i=1}^d\frac{\lambda_i(x,\theta)}{\lambda(x,\theta)}\delta_{(x,F_i(\theta))}(-)
$$
に [印付けられた点過程](Poisson.qmd#sec-marked-process) が，$Z$ の跳躍測度である．

:::

::: {.callout-note title="証明（２つの定義の等価性）" icon="false" collapse="true"}

Zig-Zag 過程に対する２つの定義を与えた．これら２つが同分布の過程を定めることは [@Corbella+2022], [@Hardcastle+2024] などさまざまなところで触れられているが，証明が必要である．

まず，$\min_{i\in[d]}T_i$ が，強度関数 $M$ が定める到着時刻に同分布であることを示す．

各 $T_i$ の密度は
$$
p_i(t)=m_i(t)e^{-M_i(t)}1_{(0,\infty)}(t)
$$
で与えられ，$T_i$ は互いに独立だから，$(T_1,\cdots,T_d)$ の結合密度もわかる．

$T_1,\cdots,T_d$ を昇順に並べた順序統計量を
$$
T_{(1)}\le\cdots\le T_{(d)}
$$
で表すとする．この $d$ 次元確率ベクトルの密度 $p$ は，
$$
p(t_1,\cdots,t_d)=1_{\Brace{t_1\le\cdots\le t_d}}(t_1,\cdots,t_d)\paren{\sum_{\sigma\in\mathfrak{S}_d}\prod_{i=1}^dm_i(t_{\sigma(i)})e^{-M_i(t_{\sigma(i)})}}
$$
と計算できる．

この $p$ を $t_2,\cdots,t_d$ に関して積分することで，$T_1$ の密度が得られる：^[計算過程は省略したが，$d=2$ の場合と，$d=3$ の場合を少しやってみると良い．]
\begin{align*}
    p_{(1)}(t)&=\int_{(0,\infty)^{d-1}}p(t_1,\cdots,t_d)\,dt_2\cdots dt_d\\
    &=\Paren{\sum_{i=1}^dm_i(t_1)}\exp\paren{-\sum_{i=1}^dM_i(t_1)}=m(t_1)e^{-M(t_1)}.
\end{align*}

これは確かに，強度関数 $m$ が定める到着時刻の密度である．

続いて，$j=\argmin_{i\in[d]}T_i$ の，$\min_{i\in[d]}T_i$ に関する条件付き確率質量関数が
$$
q(i|t)=\frac{m_i(t)}{\sum_{i=1}^dm_i(t)}
$$
であることを示す．

そのためには，任意の $i\in[d]$ と $A\in\B(\R^+)$ とに関して $\Brace{T_{(1)}\in A,T_{(1)}=T_i}$ という形の事象を計算し，密度が積の形で与えられることを見れば良い．

\begin{align*}
    &\qquad\P[T_{(1)}\in A,T_{(1)}=T_i]\\
    &=\P[T_i\in A,\forall_{j\ne i}\;T_i\le T_j]\\
    &=\int_Ap_i(t_i)\,dt_i\paren{\sum_{\sigma\in\Aut([d]\setminus\{i\})}\int^\infty_{t_i}p_{\sigma(1)}(t_{\sigma(1)})\,dt_{\sigma(1)}\int^\infty_{t_{\sigma(1)}}p_{\sigma(2)}(t_{\sigma(2)})\,dt_{\sigma(2)}\cdots\int^\infty_{t_{\sigma(d-1)}}p_{\sigma(d)}(t_{\sigma(d)})\,dt_{\sigma(d)}}\\
    &=\int_Am_i(t_i)\exp\paren{-\sum_{i=1}^dm_i(t_i)}\,dt_i\\
    &=\int_A\frac{m_i(t_i)}{m(t_i)}m(t_i)e^{-M(t_i)}\,dt_i.
\end{align*}

よって，$\min_{i\in[d]}T_i$ と $\argmin_{i\in[d]}T_i$ とに関する結合密度は，^[参照測度は，$[d]$ 上のものは計数測度 $\#$ をとっている．]
$$
q(i|t)p_{(1)}(t)
$$
という積の形で与えられることがわかった．

:::

::: {.callout-important appearance="simple" icon="false" title="まとめ"}

1. 前述の定義は，$\min_{i\in[d]}T_i$ の形で密度 $p_{(1)}$ からシミュレーションし，$\argmin_{i\in[d]}T_i$ の形で $q$ からシミュレーションしている．
2. 後述の定義は，$p_{(1)}(t)$ から直接シミュレーションし，再び $q(i|t)$ から直接シミュレーションをする．

１が２に等価であることがわかった．

:::

#### 到着時刻 $T_i$ のシミュレーション方法

Zig-Zag 過程のシミュレーションは，ほとんど強度

$$
M_i(t):=\int^t_0m_i(s)\,ds
$$
を持つ非一様 Poisson 点過程のシミュレーションに帰着される．

実はこれは，指数分布確率変数 $E_i\iidsim\Exp(1)$ について
$$
T_i\deq M_i^{-1}(E_i)
$$
と求まる．

#### Poisson 剪定

仮にこの逆関数 $M_i^{-1}$ が得られない場合でも，[剪定](PureJump.qmd#sec-simulation) [@Lewis-Shedler1979] によって $T_i$ は正確なシミュレーションが可能である．

この方法は，$M_i^{-1}$ を数値的に計算するよりも遥かに速い．これは $M_i$ の定義に積分が存在し，これが多くの場合高次元になるためである．

### レート関数の条件 {#sec-refresh-rate}

Zig-Zag 過程 $Z$ がどのような分布に従うかは，全てレート関数 $\lambda_1,\cdots,\lambda_d$ に委ねられている．

::: {.callout-tip appearance="simple" icon="false"}

Zig-Zag 過程のレート関数 $\lambda_1,\cdots,\lambda_d:E\to\R_+$ は，負の対数密度 $U\in C^1(\R^d)$ に対して，
$$
\lambda_i(x,\theta):=(\theta_i\partial_iU(x))_++\gamma_i(x,\theta_{-i})\quad(i\in[d])
$$
と定める．

ただし，次を仮定する：

* $\gamma_i:E\to\R_+$ は，$\theta_i$ のみには依らない任意の非負連続関数^[従って，レート関数 $\lambda$ は連続とする．この関数 $\gamma_i$ は，$U$ の情報には依らない追加のリフレッシュ動作を仮定に加える．実際，$\lambda_i(x,\theta)-\lambda_i(x,F_i(\theta))=\theta_i\partial_iU(x)$ である限り，$\theta$ と $F_i(\theta)$ の往来には影響を与えず釣り合っているため，どのような $\gamma_i$ をとっても，平衡分布には影響を与えない．しかし，高くするごとにアルゴリズムの対称性が上がるため，$\gamma\equiv0$ とすることが Monte Carlo 推定量の漸近分散を最小にするという [@Andrieu-Livingstone2021]．[@Bierkens+2021] でも同様の洞察がなされている．]
    $$
    \gamma_i(x,\theta)=\gamma_i(x,F_i(\theta)).
    $$
* $e^{-U}\in\L^1(\R^d)$ が成り立ち，$\pi(dx)\propt e^{-U(x)}dx$ が確率測度を定める．
* $M_i$ は $t\to\infty$ の極限で発散する：
$$
M_i(t):=\int^t_0\lambda_i(x+t\theta,\theta)\,dt
$$

:::

::: {.callout-important title="注（細かい条件たちについて）" collapse="true" icon="false"}

また，
$$
M_i(t):=\int^t_0\lambda_i(x+t\theta,\theta)\,dt
$$
は $t\to\infty$ の極限で発散する必要がある．

さもなくば，$M_i:(0,L)\to(0,\infty)\;(L\in\ocinterval{0,\infty})$ の形で定まらず，$M_i$ がこのような可微分同相を与えない場合は
$$
T_i:=M_i^{-1}(E_i),\qquad E_i\iidsim\Exp(1),
$$
によるシミュレーションも不正確になる．

:::

::: {.callout-tip title="[@Bierkens+2019 p.1294 定理2.2]" icon="false"}

上述のリフレッシュレート $\lambda_1,\cdots,\lambda_d$ に対して，定義 [-@sec-algorithm] で定まる Zig-Zag 過程 $Z$ は次の分布 $\wt{\pi}=\pi\otimes\rU(\{\pm1\}^{d})\in\cP(E)$ を不変にする：
$$
\wt{\pi}(dxd\theta)=\frac{1}{2^d}\frac{e^{-U(x)}}{\cZ}\,dxd\theta
$$

:::

::: {.callout-important title="注（拡張の可能性について）" collapse="true" icon="false"}

$\{\pm1\}^d$ 上の周辺分布が一様分布になっていること，勾配ベクトル $DU$ の情報のみを使っており，座標に沿った方向しか見ていないため $U$ の異方性に大きく左右されること，これらが「必ずしもそうある必要はない」拡張可能な点である．

:::

### エルゴード性の条件

$\pi$ が不変分布になるための十分条件 [-@sec-refresh-rate] は極めて緩かったが，MCMC として使えるためにはエルゴード性が成り立つ必要がある．

::: {.callout-tip title="[@Bierkens-Roberts2017 定理５]" icon="false"}

$d=1$ で，レート関数 $\lambda:E\to\R_+$ はある $x_0>0$ が存在して次を満たすとする：
$$
\inf_{x\ge x_0}\lambda(x,1)>\sup_{x\ge x_0}\lambda(x,-1),
$$
$$
\inf_{x\le-x_0}\lambda(x,-1)>\sup_{x\le-x_0}\lambda(x,1).
$$

このとき，ある関数 $f:E\to\cointerval{1,\infty}$ が存在して $f(x,\theta)\to\infty\;(\abs{x}\to\infty)$ が成り立ち，かつ
$$
\Norm{P^t\paren{(x,\theta),-}-\pi}_\TV\le\kappa f(x,\theta)\rho^t,\qquad(x,\theta)\in E,t\ge0,\rho\in(0,1).
$$

:::



### Subsampling Technology

Zig-Zag 過程はレート関数 $\lambda$ の設計に大きな自由度があった（第 [-@sec-refresh-rate] 節）．

これにより，Zig-Zag 過程ではバイアスを導入しない subsampling が可能であり，これを通じて **データサイズに依らない一定効率での事後分布サンプリングが可能になる** という super-efficiency [@Bierkens+2019] と呼ばれる性質を持つ．

この性質が実用上は最も重要である．詳しくは，次の記事を参照：

::: {#lst-zigzag}
:::

## シミュレーション

### １次元での例

ZigZag サンプラーは非対称なダイナミクスを持っており，その点が MALA (Metropolis-adjusted Langevin Algorithm) や HMC (Hamiltonian Monte Carlo) などの従来手法と異なる．

１次元でその違いを確認するために，Cauchy 分布という裾の重い分布を用いる．Cauchy 分布 $\rC(\mu,\sigma)$ は次のような密度を持つ：
$$
f(x)=\frac{1}{\pi\sigma}\frac{1}{1+\paren{\frac{x-\mu}{\sigma}}^2}.
$$

その裾の重さ故，平均も分散も存在しない（発散する）．

このとき，次のような観察が得られる：

::: {.callout-important appearance="simple" icon="false"}

* ZigZag サンプラーは Cauchy 分布に対して，最頻値から十分遠くから開始しても，高速に最頻値に戻ってくる．
* MALA は diffusive behaviour が見られ，最頻値に戻るまでに時間がかかる．

:::


In [ ]:
#| output: false
#| echo: false
using Plots

function plot_2dtraj(traj; title="ZigZag Sampler", save=false, label="ZigZag_2D")
    # trajからx座標とy座標を抽出
    x_coords = [state[1] for (t, state) in traj]
    y_coords = [state[2] for (t, state) in traj]

    # 軌跡のプロット
    p = plot(x_coords, y_coords, 
             label=label,
             title=title,
             xlabel="X",
             ylabel="Y",
             linewidth=2,
             marker=:circle,
             markersize=2,
             markeralpha=0.6,
             color="#78C2AD"
             #,background_color = "#F0F1EB"
             )

    # 開始点と終了点をマーク
    Plots.scatter!([x_coords[1]], [y_coords[1]], label="Start", color=:green, markersize=6)
    Plots.scatter!([x_coords[end]], [y_coords[end]], label="End", color=:red, markersize=6)

    # プロットを表示
    display(p)

    if save
        savefig(p, label * ".svg")
    end
    
end

function plot_1dtraj(traj; title="ZigZag Sampler", markersize=2, save=false, label="ZigZag_1D", ylim=nothing)
    times = [t for (t, state) in traj]
    positions = [state[1] for (t, state) in traj]

    # 軌跡のプロット
    plot_args = (
    label=label,
    title=title,
    xlabel="t",
    ylabel="X",
    linewidth=2,
    marker=:circle,
    markersize=markersize,
    markeralpha=0.6,
    color="#78C2AD"
    #,background_color = "#F0F1EB"
    )

    if ylim !== nothing
        plot_args = merge(plot_args, (ylim=ylim,))
    end

    p = plot(times, positions; plot_args...)

    if save
        savefig(p, label * ".svg")
    end

    return p
end

In [ ]:
#| code-fold: true
#| echo: false
#| output: false
using ZigZagBoomerang
const ZZB = ZigZagBoomerang

"""
    τ = freezing_time(x, θ)

computes the hitting time of a 1d particle with
constant velocity `θ` to hit 0 given the position `x`
"""
function freezing_time(x, θ)
    if θ*x > 0
        return Inf
    else
        return -x/θ
    end
end

"""
    T: Time Horizon
    (x,θ): initial state
    ∇U: gradient of the negative log-density
    c: tuning parameter
    k: weight on the Dirac measure
"""
# k determines the weight on the Dirac measure. The smaller k, the higher the wegiht
function ss_pdmp(∇U, x, θ, T, c, k, Flow::ZZB.ContinuousDynamics; adapt=false, factor=2.0, rng=Random.GLOBAL_RNG)
    t = zero(T)
    Ξ = [(t, x, θ)]
    num = acc = 0
    a, b = ZZB.ab(x, θ, c, Flow)
    t_ref = t + ZZB.waiting_time_ref(Flow)
    t′ =  t + poisson_time(a, b, rand(rng))
    tˣ = t + freezing_time(x, θ)
    while t < T
        if  tˣ < min(t_ref, t′)
            t, x, θ = ZZB.move_forward(tˣ - t, t, x, θ, Flow) # go to 0
            @assert -0.0001 < x < 0.0001 #check
            #t, x , θ = tˣ, 0.0, θ #go to 0
            push!(Ξ, (t, x, 0.0))
            t = t - log(rand(rng))/k #wait exponential time
            push!(Ξ, (t, x, θ))
            tˣ  = Inf
        elseif t_ref < t′
            t, x, θ = ZZB.move_forward(t_ref - t, t, x, θ, Flow)
            θ = sqrt(Flow.Σ)*randn(rng)
            t_ref = t +  ZZB.waiting_time_ref(Flow)
            tˣ = t + freezing_time(x, θ)
            push!(Ξ, (t, x, θ))
        else
            τ = t′ - t
            t, x, θ = ZZB.move_forward(τ, t, x, θ, Flow)
            l, lb = ZZB.λ(∇U, x, θ, Flow), ZZB.λ_bar(τ, a, b)
            num += 1
            if rand(rng)*lb < l
                acc += 1
                if l >= lb
                    !adapt && error("Tuning parameter `c` too small.")
                    c *= factor
                end
                θ = -θ  # In multi dimensions the change of velocity is different:
                        # reflection symmetric on the normal vector of the contour
                tˣ = t + freezing_time(x, θ)
                push!(Ξ, (t, x, θ))
            end
        end
        a, b = ZZB.ab(x, θ, c, Flow)
        t′ = t + poisson_time(a, b, rand(rng))
    end
    return Ξ, acc/num
end

In [ ]:
#| code-fold: true
#| echo: false
#| fig-cap: "\rC(0,1) に対する ZigZag サンプラーの軌道"
#| #| fig-label: fig-ZigZag
using ZigZagBoomerang
using LinearAlgebra
using SparseArrays
using Plots
using RandomNumbers.Xorshifts

rng = Xoroshiro128Plus(2022)

# C(0,1) 分布のポテンシャル関数（使わない）
U(x) = log(1 + x^2)

# ポテンシャル関数の勾配
∇U(x) = 2x / (1 + x^2)

# 初期値
x0 = 500.0
θ0 = 1.0

T = 2000.0  # 総シミュレーション時間

# ZigZag過程の設定
Z = ZigZag1d()

# リジェクションバウンド（必要に応じて調整）
c = 1.0

k = 1.0

# ZigZag過程の実行
trace, acc = ss_pdmp(∇U, x0, θ0, T, c, k, Z; rng=rng)

# 軌跡の離散化とサンプリング
# dt = 0.1
# traj = collect(ZigZagBoomerang.discretize(trace, dt))

traj = [(x[1], x[2]) for x in trace]

# サンプルの表示

p1 = plot_1dtraj(traj, title="1D ZigZag Sampler (Cauchy Distribution)", markersize=0, ylim=(-30, 750), label="ZigZag_1D")

MALA と比較すると，その再帰の速さが歴然としている：^[[@Bierkens+2019] にある提示の仕方である．Zig-Zag の 2000 単位時間を単純に MALA と比較はできないと筆者も考えるが，ダイナミクスに注目していただきたい．実際，自分で実装してみると，シード値をいじらないと，Zig-Zag は必ずしも 500 単位時間前後でモード $0$ に戻るわけではない．]


In [ ]:
#| code-fold: true
#| fig-cap: "\rC(0,1) に対する MALA サンプラーの軌道"
using AdvancedHMC, AdvancedMH, ForwardDiff
using LinearAlgebra
using LogDensityProblems
using LogDensityProblemsAD
using StructArrays
using Distributions
using Random

Random.seed!(2)

# Define the target distribution (1D Cauchy) using the `LogDensityProblem` interface
struct LogTargetDensityCauchy
    loc::Float64
    scale::Float64
end

LogDensityProblems.logdensity(p::LogTargetDensityCauchy, θ) = -log(π) - log(p.scale) - log(1 + ((θ[1] - p.loc)/p.scale)^2)
LogDensityProblems.dimension(p::LogTargetDensityCauchy) = 1
LogDensityProblems.capabilities(::Type{LogTargetDensityCauchy}) = LogDensityProblems.LogDensityOrder{0}()

# Choose initial parameter value for 1D
initial_θ = [500.0]

# Use automatic differentiation to compute gradients
model_with_ad = LogDensityProblemsAD.ADgradient(Val(:ForwardDiff), LogTargetDensityCauchy(0.0, 1.0))

# Set up the sampler with a multivariate Gaussian proposal.
σ² = 100
spl = MALA(x -> MvNormal((σ² / 2) .* x, σ² * I))

# Sample from the posterior.
chain = sample(model_with_ad, spl, 2000; initial_params=initial_θ, chain_type=StructArray, param_names=["θ"])

# plot
θ_vector = chain.θ
sample_values = zip(1:length(θ_vector), θ_vector)
p2 = plot_1dtraj(sample_values, title="1D MALA Sampler (Cauchy Distribution)", markersize=0, ylim=(-30, 750), label="MALA_1D")

２つを並べて比較すると，Langevin ダイナミクスの方は，少し diffusive な動き（random walk property と呼ばれる）が見られることがわかる．

::: {.callout-caution title="例（NUTS サンプラーの動き）" collapse="true" icon="false"}

NUTS サンプラーはステップサイズを極めて大きくするため，プロットによるダイナミクスの比較があまり意味を持たなくなってくる．

実際見てみると恐ろしいものである：


In [ ]:
#| echo: false
#| output: false

# Define the Cauchy distribution with location and scale
loc, scale = 0.0, 1.0
ℓπ = LogTargetDensityCauchy(loc, scale)

# Set the number of samples to draw and warmup iterations
n_samples, n_adapts = 2000, 0

# Define a Hamiltonian system
metric = DiagEuclideanMetric(1)
hamiltonian = Hamiltonian(metric, ℓπ, ForwardDiff)

# Define a leapfrog solver, with the initial step size chosen heuristically
initial_ϵ = find_good_stepsize(hamiltonian, initial_θ)
integrator = Leapfrog(initial_ϵ)

# Define an HMC sampler with the following components
#   - multinomial sampling scheme,
#   - generalised No-U-Turn criteria, and
#   - windowed adaption for step-size and diagonal mass matrix
kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

# Run the sampler to draw samples from the specified Cauchy distribution, where
#   - `samples` will store the samples
#   - `stats` will store diagnostic statistics for each sample
samples, stats = sample(hamiltonian, kernel, initial_θ, n_samples, adaptor, n_adapts; progress=true)

In [ ]:
#| echo: false
#| fig-cap: "\rC(0,1) に対する NUTS サンプラーの軌道"
using Plots

sample_values = [s[1] for s in samples]
p = plot(1:length(samples), sample_values,
             label="NUTS trajectory",
             title="1D NUTS Sampler (Cauchy distribution)",
             xlabel="t",
             ylabel="X",
             linewidth=2,
             marker=:circle,
             markersize=2,
             markeralpha=0.6,
             color="#78C2AD")

:::


### ２次元での例

::: {.callout-important appearance="simple" icon="false"}

1. 勾配 $-\nabla\log\pi$ を計算し，`∇ϕ(x, i, Γ)` の形で定義する．

:::

$$
\Sigma^{-1}=\mtrx{2}{-1}{-1}{2}
$$

で定まる分散共分散行列 $\Sigma$ を持った中心化された正規分布 $\pi(x)dx=\rN_2(0,\Sigma)(dx)$ に対しては，対数尤度は

\begin{align*}
    \log\pi(x)&=-\log\paren{(2\pi)^{d/2}(\det\Sigma)^{1/2}}\\
    &\qqquad-\frac{1}{2}x^\top\Sigma^{-1}x
\end{align*}

であるから，$\phi:=-\log\pi$ の第 $i$ 成分に関する微分は

\begin{align*}
    \partial_i\phi(x)&=\pp{}{x_i}\Paren{\frac{1}{2}x^\top\Sigma^{-1}x}\\
    &=\Sigma^{-1}x.
\end{align*}


In [ ]:
#| output: false
using ZigZagBoomerang
using SparseArrays

d = 2

# 対数尤度関数 ϕ の第 i 成分に関する微分を計算
Γ = sparse([1,1,2,2], [1,2,1,2], [2.0,-1.0,-1.0,2.0])  # <1>
∇ϕ(x, i, Γ) = ZigZagBoomerang.idot(Γ, i, x)  # <2>

# 初期値
t0 = 0.0
x0 = randn(d)
θ0 = rand([-1.0,1.0], d)

# Rejection bounds
c = 1.0 * ones(length(x0))

# ZigZag 過程をインスタンス化
Z = ZigZag(Γ, x0*0)

# シミュレーション実行
T = 20.0
zigzag_trace, (tT, xT, θT), (acc, num) = spdmp(∇ϕ, t0, x0, θ0, T, c, Z, Γ; adapt=true)

# 軌跡を離散化
traj = collect(zigzag_trace)

1. 勾配関数`∇ϕ`の計算のためには，共分散行列の逆（精度行列ともいう）を`SparseMatrixCSC`型で指定する必要があることに注意．[`idot`の実装](https://github.com/mschauer/ZigZagBoomerang.jl/blob/691afe2200fa99d762b58f5c5fd7894cc9342294/src/sparsestickyzz.jl#L42) も参照．
2. `idot`は，疎行列`Γ`の第`i`列と，疎ベクトル`x`との内積を高速に計算する関数．

::: {.callout-important title="`idot`の定義" collapse="true" icon="false"}

`idot(A,j,u)`は，疎行列`A`の第`j`列と，疎ベクトル`u`との内積を高速に計算する関数である．

```julia
function idot(A::SparseMatrixCSC, j, x)  # <1>
    rows = rowvals(A)  # <2>
    vals = nonzeros(A)  # <3>
    s = zero(eltype(x))
    @inbounds for i in nzrange(A, j)  # <4>
        s += vals[i]'*x[rows[i]][2]  # <5>
    end
    s
end
```

1. パッケージ内部で，位置 $x\in\R^d$ は全て `SparseSate` 型に統一されている？
2. 疎行列 `A` の行インデックスを取得．`rowvals(A)`はベクトルであり，第１列から順番に，非零要素のある行番号が格納されている．
3. 非零要素の値が格納されている．
4. `@inbounds` は，範囲外アクセスを許容するマクロ．高速化のためだろう．`nzrange` は，`A` の第 `j` 列に非零要素がある範囲を，第 $1$ 列から累積して何番目かで返す．すなわち，`rows[i]`で正確に第`j`列の非零要素の行番号を狙い撃ちしてイテレーションできる．
5. `x`の非零要素がある行番号 `rows[i]` における成分の値 `u[rows[i]][2]` はこのような表記になる．これと，`A` の非零要素 `vals[i]` との内積を計算．

なお，通常の行列に対しては，次のように実装されている：

```julia
idot(A, j, x) = dot((@view A[:, j]), x)
```

:::


In [ ]:
#| echo: false
plot_2dtraj(traj, title="2D ZigZag Sampler (Gaussian Distribution)"; label="ZigZag_2D")

## Zig-Zag サンプラーの実装 {.appendix}

`ZigZagBoomerang` の実装を紹介する．

Julia の MCMC パッケージ一般については次の稿を参照：

::: {#lst-julia}
:::

### ZigZag サンプラーを提供しているパッケージ一覧 {.appendix}

#### `R`パッケージ {.appendix}

[Joris Bierkens](https://diamhomes.ewi.tudelft.nl/~jorisbierkens/pdmps.html) ら開発の R パッケージ `RZigZag` ([GitHub](https://github.com/jbierkens/RZigZag) / [CRAN](https://cran.r-project.org/web/packages/RZigZag/index.html)) が最も手軽に実行できる．

#### `Julia`パッケージ {.appendix}

一方で，[@Bierkens+2023] では Julia によるパッケージ `ZigZagBoomerang`（[GitHub](https://github.com/mschauer/ZigZagBoomerang.jl) / [ANN](https://discourse.julialang.org/t/ann-zigzagboomerang-jl/57287) / [docs](https://mschauer.github.io/ZigZagBoomerang.jl/v0.1/)） も提供している．名前によらず，2022.1 月リリースの `v.0.11` 以降は BPS もサポートしている．

```julia
using Pkg
Pkg.add("ZigZagBoomerang")
```

### `ZigZag`オブジェクト {.appendix}

::: {.callout-important appearance="simple" icon="false"}

`ZigZag <: ContinuousDynamics  <: Any` は次のフィールドを持つ：^[実は６つ持つ．他の初期値は `σ=(Vector(diag(Γ))).^(-0.5); λref=0.0, ρ=0.0`]

* `Γ::SparseMatrixCSC`：ポテンシャル関数
* `μ::Vector`：平均

:::

::: {.callout-important title="注（ZigZag のメソッド）" collapse="true" icon="false"}

```julia
struct ZigZag{T,S,S2,R} <: ContinuousDynamics
    Γ::T
    μ::S
    σ::S2
    λref::R
    ρ::R
    ρ̄::R
end
ZigZag(Γ, μ, σ=(Vector(diag(Γ))).^(-0.5); λref=0.0, ρ=0.0) = ZigZag(Γ, μ, σ, λref, ρ, sqrt(1-ρ^2))
```

`dynamics.jl`にて，現在時刻にランダム時刻`τ`を加算し，位置`x`を更新するが，`θ`にはまだ触れない．

```julia
function move_forward!(τ, t, x, θ, Z::Union{BouncyParticle, ZigZag})
    t += τ
    x .+= θ .* τ
    t, x, θ
end
```

内積`∇ϕx`を用いて，位置`x`に対する第`i`成分の反射を計算する．

```julia
function reflect!(i, ∇ϕx::Number, x, θ, F::Union{ZigZag, FactBoomerang})
    θ[i] = -θ[i]
    θ
end
function reflect!(i, ∇ϕx, x, θ, F::Union{ZigZag, FactBoomerang})
    θ[i] = θ[i] - (2*dot(∇ϕx, θ[i])/normsq(∇ϕx))*∇ϕx
    θ
end
```

:::


## 文献紹介 {.appendix}

Zig-Zag Sampler を導入したのは [@Bierkens+2019] であるが，ざっと仕組みを把握をしたいならば [@Corbella+2022] の第二章がよっぽどわかりやすいだろう．
